In [1]:
#!/usr/bin/env python

"""Sample setup file"""

from __future__ import absolute_import
from __future__ import print_function

import os
import shutil
import subprocess
import numpy as np
# import mesh
import matplotlib.pyplot as plt 
import scipy.stats as stats
import decimal
# import pixiedust
#import mc_sc_stime as mc
import mc_sseg_stime as mc
from time import process_time 
#%load_ext line_profiler
#@profile


Pixiedust database opened successfully


# Method 1
Python: time loop; segment loop; constant channel variables are passed to Fortran
Fortran: Take constant variable values and then run MC for a single segment

# test input dataset

In [2]:

ntim=2;       #the number of time steps necessary for variables passed to mc module to compute successfully
nlinks=2;     #the number of links needed to define varialbe qd. ** nlinks is not used in fortran source code.
mc.var.uslinkid=1
mc.var.linkid=2

ncomp0=2; mc.var.ncomp0=ncomp0  #the number of segments of a reach upstream of the current reach
ncomp=4; mc.var.ncomp=ncomp  #the number of segments of the current reach 
mxseg=max(ncomp0,ncomp)


#MC model key arrays
#mc.var.qd=np.zeros((ntim,mxseg,nlinks))  #will store MC output qdc (flow downstream current timestep) 
mc.var.qd=np.zeros((ntim,ncomp))  #will store MC output qdc (flow downstream current timestep) 
mc.var.vela=np.zeros((ntim,ncomp)) 
mc.var.deptha=np.zeros((ntim,ncomp))


# dt
dtp=np.zeros((ncomp))
dtp[0]=60.0
dtp[1]=40.0
dtp[2]=20.0
dtp[3]=60.0
# dx
dxp=np.zeros((ncomp))
dxp[0]=1200.0
dxp[1]=2500.0
dxp[2]=2800.0
dxp[3]=1800.0
# bw
bwp=np.zeros((ncomp))
bwp[0]=50.0
bwp[1]=100.0
bwp[2]=148.0
bwp[3]=112.0
#tw
twp=np.zeros((ncomp))
twp[0]=4.0*bwp[0]
twp[1]=4.2*bwp[1]
twp[2]=4.8*bwp[2]
twp[3]=4.0*bwp[3]
# twcc
twccp=np.zeros((ncomp))
twccp[0]=twp[0]+103.2
twccp[1]=twp[1]+204.4
twccp[2]=twp[2]+275.3
twccp[3]=twp[3]+175.6
# n
npy=np.zeros((ncomp))
npy[0]= 0.03
npy[1]= 0.032
npy[2]= 0.038
npy[3]= 0.028
# ncc
nccp=np.zeros((ncomp))
nccp[0]= 1.1*npy[0]
nccp[1]= 1.2*npy[1]
nccp[2]= 1.24*npy[2]
nccp[3]= 1.12*npy[3]
# cs
csp=np.zeros((ncomp))
csp[0]= 1.2
csp[1]= 1.8
csp[2]= 2.6
csp[3]= 1.4
# so
sop=np.zeros((ncomp))
sop[0]= 0.002
sop[1]= 0.0024
sop[2]= 0.0032
sop[3]= 0.0018
#lateral flow
qlat=np.zeros((ncomp))


In [3]:
#%%pixie_debugger 

#*** WARNING: All Python variables corresponding to Fortran module must be in lowercase.
# fortran source code that is used for f2py transformation includes do k=1, ntim loop so 
# that multiple timestep computation is activated inside of the f2py module.

#run M-C model
nts=10  #the number of timestep in simulation
#variable storing all outputs in time
wqd= np.zeros((nts,mxseg))   
wvela= np.zeros((nts,mxseg)) 
wdeptha= np.zeros((nts,mxseg))

bply=nts/2.0
# uslinkflag=1/0 when upstream reach exist/not exist:
mc.var.uslinkflag=1

for k in range (0,nts):        
    # upstream reach one timestep behind flow. Note that for qd[k,i,j] 
    # k=0/1: previous/current timestep; i: node ID; j=0/1: upstream/current reach
    aply= -0.2
    #mc.var.qd[0,ncomp0-1,0]= aply*(k - bply)**2.0 - aply*bply**2.0
    mc.var.qus_prev= aply*(k - bply)**2.0 - aply*bply**2.0
    
    for i in range(0,ncomp):
        # lateral flow for current reach.
        aply= -0.8
        qlat[i]= aply*(k+1 - bply)**2.0 - aply*bply**2.0 + (i+1)*10.0
        mc.var.ql= qlat[i]
        # channel data
        mc.var.dt= dtp[i] 
        mc.var.dx= dxp[i]
        mc.var.bw= bwp[i] 
        mc.var.tw= twp[i] 
        mc.var.twcc= twccp[i]
        mc.var.n= npy[i]
        mc.var.ncc= nccp[i] 
        mc.var.cs= csp[i] 
        mc.var.so= sop[i]
        # current node
        mc.var.iseg= i+1
        # initial values
        if k==0:
            # initialize upstream reach
            #mc.var.qd[0,ncomp0-1,0]= 0.0
            mc.var.qus_prev= 0.0
            # initialize current reach
            for ii in range(0,ncomp):
                #mc.var.qd[0,i,1]= 0.0
                mc.var.qd[0,ii]= 0.0
                mc.var.vela[0,ii]= 0.0
                mc.var.deptha[0,ii]= -999.0

        # call Fortran routines
        mc.mc.main()
        
        # store computed values at time k for computation at k+1
        #qd[k,i,j]: k=0/1: previous/current timestep; i: node ID; j=0/1: upstream/current reach
        #mc.var.qd[0,i,1]= mc.var.qd[1,i,1]
        #output keeping
        wqd[k,i]= mc.var.qd[1,i]
        wvela[k,i]= mc.var.vela[1,i]
        wdeptha[k,i]= mc.var.deptha[1,i]

        
    for i in range(0,ncomp):
        mc.var.qd[0,i]= wqd[k,i]      
        mc.var.vela[0,i]= wvela[k,i]
        mc.var.deptha[0,i]= wdeptha[k,i]

    

        
#test output    
#=1
for k in range (0,nts):
    for i in range(0,ncomp):
        print(k,i,wqd[k,i])
        
#for k in range (0,nts):
#    for i in range(0,ncomp):
#        print(wvela[k,i,j])

#for k in range (0,nts):
#    for i in range(0,ncomp):
#        print(wdeptha[k,i,j])



0 0 0.13167034089565277
0 1 0.08636736869812012
0 2 0.054507412016391754
0 3 0.2452111840248108
1 0 0.6197207570075989
1 1 0.3042003810405731
1 2 0.17804938554763794
1 3 0.9608173966407776
2 0 1.4621621370315552
2 1 0.6745601892471313
2 2 0.312558114528656
2 3 2.7512083053588867
3 0 3.681124687194824
3 1 1.1542141437530518
3 2 0.45271819829940796
3 3 5.01973819732666
4 0 6.363303184509277
4 1 1.710494041442871
4 2 0.615032970905304
4 3 7.566147804260254
5 0 9.026376724243164
5 1 2.3586392402648926
5 2 0.7940337061882019
5 3 10.24357795715332
6 0 11.352397918701172
6 1 3.071197986602783
6 2 0.9818640947341919
6 3 12.88170337677002
7 0 13.077653884887695
7 1 3.803279399871826
7 2 1.1690961122512817
7 3 15.30862808227539
8 0 13.901826858520508
8 1 4.497035503387451
8 2 1.3450087308883667
8 3 17.36231231689453
9 0 13.666736602783203
9 1 5.08350944519043
9 2 1.497965931892395
9 3 18.89853858947754


# method 2

In [ ]:
ntim=2;       #the number of time steps necessary for variables passed to mc module to compute successfully
nlinks=2;     #the number of links needed to define varialbe qd. ** nlinks is not used in fortran source code.
                                    
ncomp0=2; mc.var.ncomp0=ncomp0  #the number of segments of a reach upstream of the current reach
ncomp=4; mc.var.ncomp=ncomp  #the number of segments of the current reach 
mxseg=max(ncomp0,ncomp)
mc.var.uslinkid=1
mc.var.linkid=2

#MC model outputs
mc.var.qd=np.zeros((ntim,mxseg,nlinks))  #will store MC output qdc (flow downstream current timestep) 
mc.var.vela=np.zeros((ntim,ncomp)) 
mc.var.deptha=np.zeros((ntim,ncomp))
# dt
mc.var.dt=np.zeros((ncomp))
mc.var.dt[1]=60.0
mc.var.dt[2]=40.0
mc.var.dt[3]=20.0
mc.var.dt[4]=60.0
# dx
mc.var.dx=np.zeros((ncomp))
mc.var.dx[1]=1200.0
mc.var.dx[2]=2500.0
mc.var.dx[3]=2800.0
mc.var.dx[4]=1800.0
# bw
mc.var.bw=np.zeros((ncomp))
mc.var.bw[1]=50.0
mc.var.bw[2]=100.0
mc.var.bw[3]=148.0
mc.var.bw[4]=112.0
#tw
mc.var.tw=np.zeros((ncomp))
mc.var.tw[1]=4.0*mc.var.bw[1]
mc.var.tw[2]=4.2*mc.var.bw[2]
mc.var.tw[3]=4.8*mc.var.bw[3]
mc.var.tw[4]=4.0*mc.var.bw[4]
# twcc
mc.var.twcc=np.zeros((ncomp))
mc.var.twcc[1]=mc.var.tw[1]+103.2
mc.var.twcc[2]=mc.var.tw[2]+204.4
mc.var.twcc[3]=mc.var.tw[3]+275.3
mc.var.twcc[4]=mc.var.tw[4]+175.6
# n
mc.var.n=np.zeros((ncomp))
mc.var.n[1]= 0.03
mc.var.n[2]= 0.032
mc.var.n[3]= 0.038
mc.var.n[4]= 0.028
# ncc
mc.var.ncc=np.zeros((ncomp))
mc.var.ncc[1]= 1.1*mc.var.n[1]
mc.var.ncc[2]= 1.2*mc.var.n[2]
mc.var.ncc[3]= 1.24*mc.var.n[3]
mc.var.ncc[4]= 1.12*mc.var.n[4]
# cs
mc.var.cs=np.zeros((ncomp))
mc.var.cs[1]= 1.2
mc.var.cs[2]= 1.8
mc.var.cs[3]= 2.6
mc.var.cs[4]= 1.4
# so
mc.var.so=np.zeros((ncomp))
mc.var.so[1]= 0.002
mc.var.so[2]= 0.0024
mc.var.so[3]= 0.0032
mc.var.so[4]= 0.0018
#lateral flow
mc.var.qlat=np.zeros((ncomp))

#run M-C model
nts=40  #the number of timestep in simulation
bply=nts/2.0
for k in range (0,nts):        
    # upstream reach flow. Note that for qd[k,i,j] 
    # k=0/1: previous/current timestep; i: node ID; j=0/1: upstream/current reach
    aply= -0.2
    mc.var.qd[1,ncomp0-1,0]= aply*(k+1 - bply)**2.0 - aply*bply**2.0
    # lateral flow for current reach.
    for i in range(0,ncomp):
        aply= -0.8
        mc.var.qlat[i]= aply*(k+1 - bply)**2.0 + (i+1)*10.0